In [7]:
import pandas as pd
import os
import numpy as np
import cv2 

In [2]:
df = pd.read_csv("wavfile_species_data.csv")

In [4]:
# Function to map features from df CSV file with spectrogram images
def merge_features(csv_data):
    spectrogram_folder = '/Users/insiyar/Documents/Development/Minor-2-Birdsong-Classifier/output_spectrogram_folder'
    png_filenames = []
    merged_data = csv_data.drop(columns=['filename'])
    for wav_filename in csv_data['filename']:
        # Extract common part of the filename without extension
        common_filename = os.path.splitext(wav_filename)[0]
        # Search for corresponding PNG file
        png_filename = os.path.join(spectrogram_folder, common_filename + '.png')

        if os.path.exists(png_filename):
            png_filenames.append(common_filename + '.png')
        else:
            print(f"No corresponding PNG file found for {wav_filename}")
    
    merged_data['png_filename'] = png_filenames

    return merged_data

merged_data = merge_features(df)

# Save merged data to a new CSV file
merged_data.to_csv('spectrogram_species_data.csv', index=False)

In [9]:
# Function to extract features from spectrogram images
def extract_spectrogram_features(png_filename):
    # Read the spectrogram image
    spectrogram_image = cv2.imread(png_filename, cv2.IMREAD_GRAYSCALE)

    # Calculate mean, median, standard deviation, minimum, and maximum
    mean = np.mean(spectrogram_image)
    median = np.median(spectrogram_image)
    std_dev = np.std(spectrogram_image)
    minimum = np.min(spectrogram_image)
    maximum = np.max(spectrogram_image)

    # Return the 5 features as a list
    features = [mean, median, std_dev, minimum, maximum]
    return features


In [8]:
# Load CSV data
df2 = pd.read_csv("spectrogram_species_data.csv")

In [12]:
def merge_features(csv_data):
    spectrogram_features_list = []
    spectrogram_folder = 'output_spectrogram_folder'

    for filename in csv_data['png_filename']:
        # Search for corresponding PNG file
        png_filename = os.path.join(spectrogram_folder, filename)
        if os.path.exists(png_filename):
            spectrogram_features = extract_spectrogram_features(png_filename)
            spectrogram_features_list.append(spectrogram_features)
        else:
            print(f"No corresponding PNG file found for {filename}")

    spectrogram_df = pd.DataFrame(spectrogram_features_list, columns=['mean', 'median', 'std_dev', 'minimum', 'maximum'])
    merged_data = pd.concat([csv_data, spectrogram_df], axis=1)
    return merged_data

merged_data = merge_features(df2)

# Save merged data to a new CSV file
merged_data.to_csv('Spectrogram_Features_data.csv', index=False)